# KwikSort Re-Ranking

The notebook below exemplifies how `ir_axioms` can be used to re-rank a result set in PyTerrier using the KwikSort algorithm and manually combined axioms.
We use run files and qrels from the passage retrieval task of the TREC Deep Learning track in 2019 and 2020 as example (using BM25 as a baseline).
In this notebook, we evaluate nDCG@10, reciprocal rank, and average precision for the baseline and the KwikSort-re-ranked pipeline using PyTerrier's standard `Experiment` functionality.

## Preparation

Install the `ir_axioms` framework and [PyTerrier](https://github.com/terrier-org/pyterrier). In Google Colab, we do this automatically.

In [ ]:
from sys import modules

if 'google.colab' in modules:
    !pip install -q ir_axioms[experiments] python-terrier

We initialize PyTerrier and import all required libraries and load the data from [ir_datasets](https://ir-datasets.com/).

In [ ]:
from pyterrier import started, init

if not started():
    init(tqdm="auto")

## Datasets and Index
Using PyTerrier's `get_dataset()`, we load the MS MARCO passage ranking dataset.

In [ ]:
from pyterrier.datasets import get_dataset, Dataset

# Load dataset.
dataset_name = "msmarco-passage"
dataset: Dataset = get_dataset(f"irds:{dataset_name}")
dataset_test: Dataset = get_dataset(f"irds:{dataset_name}/trec-dl-2020/judged")

Now define paths where we will store temporary files, datasets, and the search index.

In [ ]:
from pathlib import Path

cache_dir = Path("cache/")
index_dir = cache_dir / "indices" / dataset_name.split("/")[0]

If the index is not ready yet, now is a good time to create it and index the MS MARCO passages.
(Lean back and relax as this may take a while...)

In [ ]:
from pyterrier.index import IterDictIndexer

if not index_dir.exists():
    indexer = IterDictIndexer(str(index_dir.absolute()))
    indexer.index(
        dataset.get_corpus_iter(),
        fields=["text"]
    )

## Baseline Run

We use PyTerrier's `BatchRetrieve` to create a baseline search pipeline for retrieving with BM25 from the index we just created.

In [ ]:
from pyterrier.batchretrieve import BatchRetrieve

bm25 = BatchRetrieve(str(index_dir.absolute()), wmodel="BM25")

## Combine Axioms
Here we're combining many of the axioms implemented in `ir_axioms` to form a majority voting.
That is, we only want to keep preferences, where at least 50% (or 0.5) of the axioms agree.
Because some axioms require API calls or are computationally expensive, we cache the voting result using the tilde operator (`~`).

In [ ]:
from ir_axioms.axiom import (
    ArgUC, QTArg, QTPArg, aSL, PROX1, PROX2, PROX3, PROX4, PROX5, TFC1, TFC3, RS_TF, RS_TF_IDF, RS_BM25, RS_PL2, RS_QL,
    AND, LEN_AND, M_AND, LEN_M_AND, DIV, LEN_DIV, M_TDC, LEN_M_TDC, STMC1, STMC1_f, STMC2, STMC2_f, LNC1, TF_LNC, LB1,
    REG, ANTI_REG, REG_f, ANTI_REG_f, ASPECT_REG, ASPECT_REG_f, ORIG, VoteAxiom
)

axiom = (
        ~VoteAxiom([
            ArgUC(), QTArg(), QTPArg(), aSL(),
            LNC1(), TF_LNC(), LB1(),
            PROX1(), PROX2(), PROX3(), PROX4(), PROX5(),
            REG(), REG_f(), ANTI_REG(), ANTI_REG_f(), ASPECT_REG(), ASPECT_REG_f(),
            AND(), LEN_AND(), M_AND(), LEN_M_AND(), DIV(), LEN_DIV(),
            RS_TF(), RS_TF_IDF(), RS_BM25(), RS_PL2(), RS_QL(),
            TFC1(), TFC3(), M_TDC(), LEN_M_TDC(),
            STMC1(), STMC1_f(), STMC2(), STMC2_f(),
        ], minimum_votes=0.5) | ORIG()
)

## KwikSort Re-ranking
After having defined the axiom to use for re-ranking, we create a new PyTerrier pipeline that re-ranks the top-20 baseline results using the KwikSort algorithm.
KwikSort works similar to Quicksort, but instead of comparing items by natural order, it compares by axiom preferences.

In [ ]:
from ir_axioms.modules.pivot import MiddlePivotSelection
from ir_axioms.backend.pyterrier.transformers import KwikSortReranker

kwiksort = bm25 % 20 >> KwikSortReranker(
    axiom=axiom,
    index=index_dir,
    dataset=dataset_name,
    pivot_selection=MiddlePivotSelection(),
    cache_dir=cache_dir,
    verbose=True
)

## Experimental Evaluation
Because our axiomatic re-rankers are PyTerrier modules, we can now use PyTerrier's `Experiment` interface to evaluate various metrics and to compare our new approach to the BM25 baseline ranking.
Refer to the PyTerrier [documentation](https://pyterrier.readthedocs.io/en/latest/experiments.html) to learn more about running experiments.
(We concatenate results from the Baseline ranking for the ranking positions after the top-20 using the `^` operator.)

In [ ]:
from pyterrier.pipelines import Experiment
from ir_measures import nDCG, MAP, RR

experiment = Experiment(
    [bm25, kwiksort ^ bm25],
    dataset_test.get_topics(),
    dataset_test.get_qrels(),
    [nDCG @ 10, RR, MAP],
    ["BM25", "KwikSort"],
    verbose=True,
)
experiment.sort_values(by="nDCG@10", ascending=False, inplace=True)


In [ ]:
experiment